In [ ]:
# Real-Time Hand Proximity Alert System Using OpenCV (No ML Models)

In [1]:
import cv2
import numpy as np

# -----------------------------------
# 1. Start Webcam
# -----------------------------------
cap = cv2.VideoCapture(0)

ret, frame = cap.read()
if not ret:
    print("Could not open camera.")
    exit()

# -----------------------------------
# 2. Select object ROI manually
# -----------------------------------
roi = cv2.selectROI("Select Object", frame, showCrosshair=True, fromCenter=False)
cv2.destroyWindow("Select Object")

x, y, w, h = roi
vx1, vy1 = x, y
vx2, vy2 = x + w, y + h   

obj_center = (x + w // 2, y + h // 2)



# For smoothing + movement detection
last_hand_center = None

def detect_hand(frame):
    """ Returns (hand_center, contour) or (None, None) """

    # Convert to HSV (works very well for skin detection)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Skin color range (generalized, adjustable for your lighting)
    lower = np.array([0, 30, 60])
    upper = np.array([20, 150, 255])

    mask = cv2.inRange(hsv, lower, upper)

    # Reduce noise
    mask = cv2.GaussianBlur(mask, (7, 7), 0)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None, None

    # Select largest contour (assumed hand)
    c = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(c)

    if area < 3000:
        return None, None

    # Compute center of contour
    M = cv2.moments(c)
    if M['m00'] == 0:
        return None, None

    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])

    return (cx, cy), c



# -----------------------------------
# 3. MAIN LOOP (With State Logic)
# -----------------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    hand_center, hand_contour = detect_hand(frame)

    # Detect movement of hand
    if hand_center is not None:
        cv2.circle(frame, hand_center, 8, (0, 0, 255), -1)

        if last_hand_center is not None:
            movement = np.linalg.norm(np.array(hand_center) - np.array(last_hand_center))
            if movement > 12:
                hand_moving = True

        last_hand_center = hand_center

        if hand_contour is not None:
            cv2.drawContours(frame, [hand_contour], -1, (0, 255, 255), 2)


    
    # ----------------------------
    # Distance-based State Logic
    # ----------------------------
    state = "SAFE"
    color = (0, 255, 0)

    if hand_center is not None:
        dist = int(np.linalg.norm(np.array(hand_center) - np.array(obj_center)))

        if dist > 120:
            state = "SAFE"
            color = (0, 255, 0)
        elif dist <= 120 and dist > 60:
            state = "WARNING"
            color = (0, 255, 255)
        else:
            state = "DANGER"
            color = (0, 0, 255)

        cv2.putText(frame, f"Distance: {dist}px", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    
    
    # -----------------------------------
    # Draw Virtual Object (Color-coded)
    # -----------------------------------
    cv2.rectangle(frame, (vx1, vy1), (vx2, vy2), color, 3)
    cv2.circle(frame, obj_center, 6, color, -1)

    # -----------------------------------
    # Display State
    # -----------------------------------
    cv2.putText(frame, state, (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

    if state == "DANGER":
        cv2.putText(frame, "DANGER DANGER", (80, 130),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 4)


    
    # -----------------------------------
    # Show Frame
    # -----------------------------------
    cv2.imshow("Hand Tracking POC", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()